## Imports

In [1]:
import os
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf

from collections.abc import Iterable

import faiss
from faiss import write_index, read_index

from sentence_transformers import SentenceTransformer

## Code to Sentencize Text

In [2]:
def process_documents(documents: Iterable[str],
                      document_ids: Iterable,
                      split_sentences: bool = True,
                      filter_len: int = 3,
                      disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Main helper function to process documents from the EMR.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param document_type: String denoting the document type to be processed
    :param document_sections: List of sections for a given document type to process
    :param split_sentences: Flag to determine whether to further split sections into sentences
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """
    
    df = sectionize_documents(documents, document_ids, disable_progress_bar)

    if split_sentences:
        df = sentencize(df.text.values, 
                        df.document_id.values,
                        df.offset.values, 
                        filter_len, 
                        disable_progress_bar)
    return df


def sectionize_documents(documents: Iterable[str],
                         document_ids: Iterable,
                         disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Obtains the sections of the imaging reports and returns only the 
    selected sections (defaults to FINDINGS, IMPRESSION, and ADDENDUM).

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param disable_progress_bar: Flag to disable tqdm progress bar
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `offset`
    """
    processed_documents = []
    for document_id, document in tqdm(zip(document_ids, documents), total=len(documents), disable=disable_progress_bar):
        row = {}
        text, start, end = (document, 0, len(document))
        row['document_id'] = document_id
        row['text'] = text
        row['offset'] = (start, end)

        processed_documents.append(row)

    _df = pd.DataFrame(processed_documents)
    if _df.shape[0] > 0:
        return _df.sort_values(['document_id', 'offset']).reset_index(drop=True)
    else:
        return _df


def sentencize(documents: Iterable[str],
               document_ids: Iterable,
               offsets: Iterable[tuple[int, int]],
               filter_len: int = 3,
               disable_progress_bar: bool = False) -> pd.DataFrame:
    """
    Split a document into sentences. Can be used with `sectionize_documents`
    to further split documents into more manageable pieces. Takes in offsets
    to ensure that after splitting, the sentences can be matched to the
    location in the original documents.

    :param documents: Iterable containing documents which are strings
    :param document_ids: Iterable containing document unique identifiers
    :param offsets: Iterable tuple of the start and end indices
    :param filter_len: Minimum character length of a sentence (otherwise filter out)
    :return: Pandas DataFrame containing the columns `document_id`, `text`, `section`, `offset`
    """

    document_sentences = []
    for document, document_id, offset in tqdm(zip(documents, document_ids, offsets), total=len(documents), disable=disable_progress_bar):
        try:
            _, sentence_offsets = bf.text_to_sentences_and_offsets(document)
            for o in sentence_offsets:
                if o[1]-o[0] > filter_len:
                    sentence = document[o[0]:o[1]]
                    abs_offsets = (o[0]+offset[0], o[1]+offset[0])
                    row = {}
                    row['document_id'] = document_id
                    row['text'] = sentence
                    row['offset'] = abs_offsets
                    document_sentences.append(row)
        except:
            continue
    return pd.DataFrame(document_sentences)

## Configurations

In [3]:
MODEL = '/home/clay/research/kaggle/kaggle_llm/data/sentence_transformer_model'
DEVICE = 0
MAX_LENGTH = 384
BATCH_SIZE = 16

## Load Data

In [15]:
WIKI_PATH = "/home/clay/research/kaggle/kaggle_llm/data/wikipedia-20230701/"
wiki_files = os.listdir(WIKI_PATH)

In [4]:
trn = pd.read_csv("/home/clay/research/kaggle/kaggle_llm/data/kaggle-llm-science-exam/train.csv")

In [6]:
model = SentenceTransformer(MODEL, device='cuda')
model.max_seq_length = MAX_LENGTH
model = model.half()

#### Using precomputed index of the Wikipedia 2023-07 dump

Dataset can be found: https://www.kaggle.com/datasets/jjinho/wikipedia-2023-07-faiss-index

In [7]:
sentence_index = read_index("/home/clay/research/kaggle/kaggle_llm/data/wikipedia_index/wikipedia_202307.index")

## Encode the prompts

We observe that the prompts contain the subject matter almost always at the end. Here we just encode the entire prompt using `sentence_transformers` so that we can use semantic search to find appropriate articles that may have information relating to the questions.

In [8]:
prompt_embeddings = model.encode(
    trn.prompt.values, 
    batch_size=BATCH_SIZE, 
    device=DEVICE, 
    show_progress_bar=True, 
    convert_to_tensor=True, 
    normalize_embeddings=True
).half()
prompt_embeddings = prompt_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [9]:
_ = gc.collect()

In [10]:
## Get the top 3 pages that are likely to contain the topic of interest
search_score, search_index = sentence_index.search(prompt_embeddings, 3)

In [11]:
## Save memory - delete sentence_index since it is no longer necessary
del sentence_index
del prompt_embeddings
_ = gc.collect()

#### Load the Wikipedia Index File

In [12]:
df = pd.read_parquet("/home/clay/research/kaggle/kaggle_llm/data/wikipedia-20230701/wiki_2023_index.parquet", columns=['id', 'file'])

In [13]:
## Get the article and associated file location using the index
wikipedia_file_data = []

for i, (scr, idx) in tqdm(enumerate(zip(search_score, search_index)), total=len(search_score)):
    
    ## Get indices by score threshold
    #scr_idx = idx[np.where(scr <= 0.85)]
    scr_idx = idx
    _df = df.loc[scr_idx].copy()
    _df['prompt_id'] = i
    wikipedia_file_data.append(_df)
wikipedia_file_data = pd.concat(wikipedia_file_data).reset_index(drop=True)
wikipedia_file_data = wikipedia_file_data[['id', 'prompt_id', 'file']].drop_duplicates().sort_values(['file', 'id']).reset_index(drop=True)

## Save memory - delete df since it is no longer necessary
del df
_ = gc.collect()

  0%|          | 0/200 [00:00<?, ?it/s]

In [16]:
## Get the full text data
wiki_text_data = []

for file in tqdm(wikipedia_file_data.file.unique(), total=len(wikipedia_file_data.file.unique())):
    _id = [str(i) for i in wikipedia_file_data[wikipedia_file_data['file']==file]['id'].tolist()]
    _df = pd.read_parquet(f"{WIKI_PATH}/{file}", columns=['id', 'text'])

    _df = _df[_df['id'].isin(_id)]
    wiki_text_data.append(_df)
    _ = gc.collect()
wiki_text_data = pd.concat(wiki_text_data).drop_duplicates().reset_index(drop=True)
_ = gc.collect()

  0%|          | 0/28 [00:00<?, ?it/s]

## Split full-text Wikipedia Documents into Sentences

We split the Wikipedia documents into sentences because we can observe that in many cases it seems that GPT3.5 directly took the answers from the text. We therefore want to retrieve the most similar sentences to provide context.

In [17]:
## Parse documents into sentences
processed_wiki_text_data = process_documents(wiki_text_data.text.values, wiki_text_data.id.values)

  0%|          | 0/576 [00:00<?, ?it/s]

  0%|          | 0/576 [00:00<?, ?it/s]

In [18]:
## Get embeddings of the wiki text data
wiki_data_embeddings = model.encode(
    processed_wiki_text_data.text, 
    batch_size=BATCH_SIZE, device=DEVICE, 
    show_progress_bar=True, 
    convert_to_tensor=True, 
    normalize_embeddings=True
).half()
wiki_data_embeddings = wiki_data_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/2020 [00:00<?, ?it/s]

In [19]:
_ = gc.collect()

Found that encoding the prompt and the answers gave the best quality results for retrieval.

In [20]:
## Combine all answers
trn['answer_all'] = trn.apply(lambda x: " ".join([x['A'], x['B'], x['C'], x['D'], x['E']]), axis=1)

## Search using the prompt and answers to guide the search
trn['prompt_answer_stem'] = trn['prompt'] + " " + trn['answer_all']

In [21]:
question_embeddings = model.encode(trn.prompt_answer_stem.values, batch_size=BATCH_SIZE, device=DEVICE, show_progress_bar=True, convert_to_tensor=True, normalize_embeddings=True).half()
question_embeddings = question_embeddings.detach().cpu().numpy()

Batches:   0%|          | 0/13 [00:00<?, ?it/s]

In [22]:
## Parameter to determine how many relevant sentences to include
NUM_SENTENCES_INCLUDE = 3

## List containing Question, Choices, Context
prompt_contexts = []

## List containing just Context
contexts = []

for r in trn.itertuples():
    prompt_context = ""

    prompt_id = r.id

    prompt_indices = processed_wiki_text_data[processed_wiki_text_data['document_id'].isin(wikipedia_file_data[wikipedia_file_data['prompt_id']==prompt_id]['id'].values)].index.values
    prompt_context += "Question: " + trn.prompt.iloc[prompt_id] + "\n"

    prompt_context += "Choices:\n"
    prompt_context += "(A) " + trn.A.iloc[prompt_id] + "\n"
    prompt_context += "(B) " + trn.B.iloc[prompt_id] + "\n"
    prompt_context += "(C) " + trn.C.iloc[prompt_id] + "\n"
    prompt_context += "(D) " + trn.D.iloc[prompt_id] + "\n"
    prompt_context += "(E) " + trn.E.iloc[prompt_id] + "\n"

    if prompt_indices.shape[0] > 0:
        prompt_context += "Context:\n"
        ## Per Prompt Index
        prompt_index = faiss.index_factory(wiki_data_embeddings.shape[1], "Flat")
        prompt_index.add(wiki_data_embeddings[prompt_indices])

        context = ""
        
        ## Get the top matches
        ss, ii = prompt_index.search(question_embeddings, NUM_SENTENCES_INCLUDE)
        for _s, _i in zip(ss[prompt_id], ii[prompt_id]):
            ## Threshold on the score
            if _s < 2:
                context += processed_wiki_text_data.loc[prompt_indices]['text'].iloc[_i] + "\n"
        prompt_context += context
        
    contexts.append(context)
    prompt_contexts.append(prompt_context)

In [23]:
trn['context'] = contexts

In [ ]:
trn.to_csv("./train_context.csv", index=False)

## Open Book Test Taking!

Below we can see some results which provides not only the question and choices, but also context from Wikipedia which may provide crucial hints or even the answers themselves!

In [24]:
for i, p in enumerate(prompt_contexts[:10]):
    print(f"Question {i}")
    print(p)
    print()

Question 0
Question: Which of the following statements accurately describes the impact of Modified Newtonian Dynamics (MOND) on the observed "missing baryonic mass" discrepancy in galaxy clusters?
Choices:
(A) MOND is a theory that reduces the observed missing baryonic mass in galaxy clusters by postulating the existence of a new form of matter called "fuzzy dark matter."
(B) MOND is a theory that increases the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 20.
(C) MOND is a theory that explains the missing baryonic mass in galaxy clusters that was previously considered dark matter by demonstrating that the mass is in the form of neutrinos and axions.
(D) MOND is a theory that reduces the discrepancy between the observed missing baryonic mass in galaxy clusters and the measured velocity dispersions from a factor of around 10 to a factor of about 2.
(E) MOND is a theory that 